In [ ]:
## This is a basic CVXPY based implementation on a toy dataset for the paper
## "Neural Networks are Convex Regularizers: Exact Polynomial-time Convex Optimization Formulations for Two-layer Networks"
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import torch

In [ ]:
def relu(x):
    return np.maximum(0,x)
def drelu(x):
    return x>=0
n=5 # number of samples
d=2 # dimension of the data
beta=1e-3 # regularization parameter

In [ ]:
seed_value = 42
torch.manual_seed(seed_value)  # Set the seed for PyTorch

In [ ]:
# 1. Generate random dataset with n samples

X = torch.rand(n, 2) * 10 - 5 # Random 2D input in range [-5, 5)
# Create Y based on the first element of each row in X

In [ ]:
X=X.numpy()
Y=Y.numpy()

In [ ]:
##Check
print(X.shape)
print(Y.shape)
print(type(X))
print(type(Y))
print(X[:10])
print(Y[:10])

(5, 2)
(5,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[-0.58865356 -0.42498016]
 [-3.0856812  -0.20347166]
 [-3.047759   -1.9955232 ]
 [-3.7171376  -1.0317934 ]
 [-0.29614258 -4.33407   ]]
[-1 -1 -1 -1  1]


In [ ]:
dmat=np.empty((n,0))

In [ ]:
print(dmat)
print(dmat.shape)

[]
(5, 0)


In [ ]:
## Finite approximation of all possible sign patterns
for i in range(int(1e5)):
    u=np.random.randn(d,1)
    #u is a 2D array of shape (d, 1) filled with random floats sampled from a standard normal distribution (mean = 0, standard deviation = 1)
    dmat=np.append(dmat,drelu(np.dot(X,u)),axis=1)
    #The function appends the result of drelu(np.dot(X, u)) as a new column to the right of the dmat array.

dmat=(np.unique(dmat,axis=1))

In [ ]:
print(dmat)

[[0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 0. 1. 1. 1. 1.]
 [0. 0. 0. 1. 1. 0. 0. 1. 1. 1.]
 [0. 1. 0. 0. 0. 1. 1. 1. 0. 1.]]


In [ ]:
# Optimal CVX
m1=dmat.shape[1] #m1=number of columns in dmat
Uopt1=cp.Variable((d,m1))
Uopt2=cp.Variable((d,m1))

In [ ]:
#A symbolic representation of a quantity in Python refers to a mathematical expression or object that represents a value without specifying the actual value.
# Instead of directly working with numbers, symbolic representations allow you to manipulate, simplify, and solve mathematical expressions symbolically.
print(Uopt1)
print(m1)
print(Uopt1.shape)

var205
10
(2, 10)


In [ ]:
## Below we use hinge loss as a performance metric for binary classification
yopt1=cp.Parameter((n,1))
yopt2=cp.Parameter((n,1))
#You would use it to represent a fixed vector that doesn't change during the optimization, but its values can be set before or during the optimization.

In [ ]:
yopt1=cp.sum(cp.multiply(dmat,(X*Uopt1)),axis=1)
yopt2=cp.sum(cp.multiply(dmat,(X*Uopt2)),axis=1)

/usr/local/lib/python3.11/dist-packages/cvxpy/expressions/expression.py:667: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 9 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/cvxpy/expressions/expression.py:667: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 10 times so far.

  warnings.warn(msg, UserWarning)


In [ ]:
cost=cp.sum((Y - (yopt1 - yopt2))**2)/n + (beta/2)*(cp.norm(Uopt1, 'fro')**2 + cp.norm(Uopt2, 'fro')**2)

In [ ]:
constraints=[]
constraints+=[cp.multiply((2*dmat-np.ones((n,m1))),(X*Uopt1))>=0]
constraints+=[cp.multiply((2*dmat-np.ones((n,m1))),(X*Uopt2))>=0]

/usr/local/lib/python3.11/dist-packages/cvxpy/expressions/expression.py:667: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 11 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/cvxpy/expressions/expression.py:667: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 12 times so far.

  warnings.warn(msg, UserWarning)


In [ ]:
prob=cp.Problem(cp.Minimize(cost),constraints)
prob.solve()
cvx_opt=prob.value

In [ ]:
print("Convex program objective value: ",cvx_opt)

Convex program objective value (eq (8)):  0.06956456608678105
